In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [5]:
gdf_building = gpd.read_file('../../data/processed/building/building_manhattan_commercial_street_traffic_density_other.geojson')

In [6]:
gdf_building.head()

,height,mpluto_bbl,cnstrct_yr,globalid,bin,office_area,retail_area,residential_area,OBJECTID,street_width_min,...,idw_aadt_2021,idw_atvc_2018,idw_atvc_2019,idw_atvc_2020,office_within_450ft,retail_within_450ft,residential_within_450ft,distance_to_park,distance_to_school,geometry
0,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},1062896,0,2250,10344,100007,60.0,...,17083.134233,110.061311,140.829357,332.410589,5550,59662,898416,79.391247,144.771475,"MULTIPOLYGON (((1001211.761 244524.798, 100123..."
1,155.49,1008870001,1935,{0B06F7A8-1F83-44F3-97DD-AC524A6374CA},1018457,50018,4000,0,101045,32.0,...,11495.144080,129.010552,178.320105,272.446791,2550406,351887,1164766,727.736846,544.997559,"MULTIPOLYGON (((989135.970 210787.603, 989181...."
2,167.21446831,1013490035,1931,{A091951D-C73E-4B77-9A61-716C666C5446},1039988,2890,0,70110,99804,60.0,...,16299.638335,151.952815,174.349496,240.654167,170032,135993,3123941,619.229208,848.838949,"MULTIPOLYGON (((994153.480 215659.003, 994165...."
3,28.57,1010610006,1910,{2BE9F674-67E7-4A49-8535-F636A16D8DA5},1026714,0,0,0,103068,32.0,...,11238.595524,163.402973,182.997568,237.267362,228739,75352,1677852,363.174879,326.217620,"MULTIPOLYGON (((986858.328 218070.430, 986839...."
4,73.13,1003060013,1900,{2403088E-77A6-4071-8B5E-D54AFBC720D9},1003956,0,3800,0,92425,34.0,...,8351.143163,105.344303,137.101029,350.242494,99564,185023,676931,73.236227,490.773688,"MULTIPOLYGON (((986180.396 200723.234, 986187...."


In [7]:
df_restaurant = pd.read_csv('../../data/processed/restaurants_coord.csv')
points = [Point(x, y) for x, y in zip(df_restaurant['longitude'], df_restaurant['latitude'])]
gdf_restaurants = gpd.GeoDataFrame(df_restaurant, geometry=points, crs=4326).to_crs(gdf_building.crs)

In [8]:
def count_restaurants_within_buffer(building_geometry, buffer_distance=328):
    '''
    input: building_geometry (shapely Polygon or MultiPolygon), buffer_distance (float)
    output: restaurant_count (int)
    '''
    # create a buffer zone around the building
    buffer_zone = building_geometry.buffer(buffer_distance)
    # find all restaurants that are within the buffer zone
    intersecting_restaurants = gdf_restaurants.loc[gdf_restaurants.within(buffer_zone)]
    # count the number of restaurants within the buffer zone
    restaurant_count = len(intersecting_restaurants)
    return restaurant_count

In [9]:
gdf_building.loc[:,'food_100'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=328)
gdf_building.loc[:,'food_400'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=1312)
gdf_building.loc[:,'food_800'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=2624)
gdf_building.loc[:,'food_1000'] = gdf_building['geometry'].apply(count_restaurants_within_buffer, buffer_distance=3280)

In [10]:
gdf_building.loc[:,'idw_aadt_mean'] = gdf_building.loc[:,['idw_aadt_2019', 'idw_aadt_2020', 'idw_aadt_2021']].mean(axis=1)
gdf_building.loc[:,'idw_atvc_mean'] = gdf_building.loc[:,['idw_atvc_2018','idw_atvc_2019', 'idw_atvc_2020']].mean(axis=1)

In [11]:
gdf_building.drop(['mpluto_bbl',
                   'globalid',
                   'station_complex_id',
                   'ridership_late_night',
                   'ridership_night',
                   'betweeness',
                   'street_width_min',
                   'posted_speed',
                   'idw_aadt_2019', 'idw_aadt_2020', 'idw_aadt_2021',
                   'idw_atvc_2018','idw_atvc_2019', 'idw_atvc_2020',
                   'OBJECTID'], axis=1).to_crs(4326).to_file('../../data/processed/building/building_240417.geojson', driver='GeoJSON')

In [12]:
gdf_building.columns

Index(['height', 'mpluto_bbl', 'cnstrct_yr', 'globalid', 'bin', 'office_area',
       'retail_area', 'residential_area', 'OBJECTID', 'street_width_min',
       'street_width_max', 'posted_speed', 'betweeness', 'station_complex_id',
       'ridership_evening', 'ridership_late_night', 'ridership_midday',
       'ridership_morning', 'ridership_night', 'distance_from_station(ft)',
       'idw_aadt_2019', 'idw_aadt_2020', 'idw_aadt_2021', 'idw_atvc_2018',
       'idw_atvc_2019', 'idw_atvc_2020', 'office_within_450ft',
       'retail_within_450ft', 'residential_within_450ft', 'distance_to_park',
       'distance_to_school', 'geometry', 'food_100', 'food_400', 'food_800',
       'food_1000', 'idw_aadt_mean', 'idw_atvc_mean'],
      dtype='object')

In [13]:
gdf_building

,height,mpluto_bbl,cnstrct_yr,globalid,bin,office_area,retail_area,residential_area,OBJECTID,street_width_min,...,residential_within_450ft,distance_to_park,distance_to_school,geometry,food_100,food_400,food_800,food_1000,idw_aadt_mean,idw_atvc_mean
0,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},1062896,0,2250,10344,100007,60.0,...,898416,79.391247,144.771475,"MULTIPOLYGON (((1001211.761 244524.798, 100123...",3,30,52,60,15365.929020,194.433752
1,155.49,1008870001,1935,{0B06F7A8-1F83-44F3-97DD-AC524A6374CA},1018457,50018,4000,0,101045,32.0,...,1164766,727.736846,544.997559,"MULTIPOLYGON (((989135.970 210787.603, 989181....",11,140,418,550,10614.511164,193.259149
2,167.21446831,1013490035,1931,{A091951D-C73E-4B77-9A61-716C666C5446},1039988,2890,0,70110,99804,60.0,...,3123941,619.229208,848.838949,"MULTIPOLYGON (((994153.480 215659.003, 994165....",5,62,201,289,15061.037497,188.985493
3,28.57,1010610006,1910,{2BE9F674-67E7-4A49-8535-F636A16D8DA5},1026714,0,0,0,103068,32.0,...,1677852,363.174879,326.217620,"MULTIPOLYGON (((986858.328 218070.430, 986839....",6,58,270,349,10593.797147,194.555968
4,73.13,1003060013,1900,{2403088E-77A6-4071-8B5E-D54AFBC720D9},1003956,0,3800,0,92425,34.0,...,676931,73.236227,490.773688,"MULTIPOLYGON (((986180.396 200723.234, 986187....",12,144,439,506,7805.773638,197.562609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37404,26.00608505,1003060022,2012,{BCEF4A7B-9A3B-45DE-BEBD-64809EAA3861},1077601,5000,5000,0,95865,40.0,...,695434,198.086253,493.914143,"MULTIPOLYGON (((986313.302 200636.585, 986311....",10,160,441,508,8203.260861,196.818669
37405,35.44835474,1016280057,1923,{B072371A-F903-41B1-B7AF-DA0BC4835B50},1051816,0,0,0,99663,30.0,...,1204574,12.371509,946.896800,"MULTIPOLYGON (((998256.785 226794.479, 998271....",3,31,86,124,13001.504346,181.910330
37406,338.0,1000920030,2023,{5CDE3269-D9A3-4732-BBB0-AD92B46BDDF2},1091002,0,0,0,99112,20.0,...,1954447,300.930018,261.005278,"MULTIPOLYGON (((982522.527 198275.305, 982453....",8,67,212,339,6479.796892,210.566177
37407,84.72,1000920032,1925,{5D66351B-DDFF-43FB-A05D-E5A3E861CE1B},1001278,23039,2000,0,112560,22.0,...,2003782,410.186751,262.754254,"MULTIPOLYGON (((982550.394 198238.001, 982484....",8,67,213,338,8440.579671,211.151782
